# Extract tree shapes

The input for this notebok is the pre-processed and filtered cloud that results from notebook `1. Tree filter.ipynb`. From this data we extract clusters of points that form potential trees and tree-clusters. The location, surface area and volume of the resulting clusters are extracted.

In [ ]:
import set_path

import numpy as np
import laspy
import pathlib
import pickle
import os
import geopandas as gpd
import shapely.geometry as sg
from scipy.spatial import ConvexHull
from tqdm.notebook import tqdm

from upcp.region_growing.label_connected_comp import LabelConnectedComp
from upcp.utils import clip_utils

import gvl.alpha_shape_utils as as_utils
import gvl.helper_functions as utils

In [ ]:
import warnings  # temporary, to suppress runtime warnings from alpha_shape

warnings.filterwarnings("ignore")

## Settings

In [ ]:
DATA_FOLDER = pathlib.Path("../data")

input_dir = DATA_FOLDER / "ahn4_trees"
output_dir = DATA_FOLDER / "shapes"
N = 4  # Number of digits in tilecode format
TILE_WIDTH = 50  # Tile width in meters

tmp_file = output_dir / "data_tmp.pickle"
resume = True

output_csv = output_dir / "trees.csv"
output_shp = output_dir / "trees.shp"

# Settings for connected component clustering.
tree_lcc = {"grid_size": 0.5, "min_component_size": 50}

# Optional: compute concave (alpha shape) hull. This is slower, but more precice.
use_concave = True
# Minimum area for which to compute concave hull. For smaller areas the convex hull will be used.
concave_min_area = 10.0
# Alpha determines the "concaveness" of the resulting shape, with 0 being convex.
alpha = 1.75

# Downsampling settings, reduces complexity of concave hull operations.
downsample_n = (
    10  # Number of points in a cluster after which downsampling will be applied.
)
downsample_voxelsize = 0.25

# Our classification
UNKNOWN = 0
TREE = 1
NOISE = 2
OTHER = 0

In [ ]:
# Create output folder
output_dir.mkdir(parents=True, exist_ok=True)

## Main loop

In [ ]:
input_files = list(input_dir.glob("trees*.laz"))

In [ ]:
# Check if tmp_file exists.
if resume and os.path.exists(tmp_file):
    with open(tmp_file, "rb") as f:
        data = pickle.load(f)
        existing_codes = set(data["tilecode"])
        input_files = [
            file
            for file in input_files
            if utils.get_tilecode_from_filename(file.name, n_digits=N)
            not in existing_codes
        ]
else:
    data = {
        "tilecode": [],
        "n_points": [],
        "convex_hull": [],
        "concave_hull": [],
        "location": [],
    }

In [ ]:
pbar = tqdm(input_files, unit="file", smoothing=0)

for file in pbar:
    tilecode = utils.get_tilecode_from_filename(file.name, n_digits=N)
    pbar.set_postfix_str(tilecode)

    # Load LAS data
    las = laspy.read(file)
    mask = las.label == TREE
    if np.count_nonzero(mask) == 0:
        continue
    points_xyz = np.vstack((las.x[mask], las.y[mask], las.z[mask])).T

    # Extract "tree" clusters
    lcc = LabelConnectedComp(
        grid_size=tree_lcc["grid_size"],
        min_component_size=tree_lcc["min_component_size"],
    )
    point_components = lcc.get_components(points_xyz)

    cc_labels = np.unique(point_components)
    cc_labels = set(cc_labels).difference((-1,))

    # Iterate over the clusters
    for cc in tqdm(cc_labels, smoothing=0, leave=False):
        # select points that belong to the cluster
        cc_mask = point_components == cc
        cc_points = points_xyz[cc_mask, :2]

        if np.count_nonzero(cc_mask) > downsample_n:
            cc_points = utils.voxel_downsample(cc_points, downsample_voxelsize)

        convex_poly = sg.Polygon(
            cc_points[ConvexHull(cc_points, qhull_options="QJ").vertices]
        )
        if (not use_concave) or (convex_poly.area < concave_min_area):
            concaves = [convex_poly]
        else:
            edges = as_utils.alpha_shape(cc_points, alpha=alpha, only_outer=True)
            concaves = as_utils.generate_poly_from_edges(edges, cc_points)

        for hull in concaves:
            hull_mask = clip_utils.poly_clip(cc_points, hull)
            data["tilecode"].append(tilecode)
            data["n_points"].append(np.count_nonzero(hull_mask))
            data["convex_hull"].append(convex_poly)
            data["concave_hull"].append(hull)
            data["location"].append(sg.Point(np.mean(cc_points[hull_mask], axis=0)))

    with open(tmp_file, "wb") as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Save results as CSV

In [ ]:
if use_concave:
    tree_gdf = gpd.GeoDataFrame(data=data, geometry="concave_hull", crs="epsg:28992")
else:
    del data["concave_hull"]
    tree_gdf = gpd.GeoDataFrame(data=data, geometry="convex_hull", crs="epsg:28992")

In [ ]:
tree_gdf.to_csv(output_csv, index=False)

In [ ]:
# Optional: save as .shp file with only convex / concave hulls.
if use_concave:
    tree_gdf_2 = tree_gdf[["tilecode", "n_points", "concave_hull"]]
else:
    tree_gdf_2 = tree_gdf[["tilecode", "n_points", "convex_hull"]]
tree_gdf_2.to_file(output_shp)

## Plotting (optional)

In [ ]:
# To load, use this snippet
import geopandas as gpd
from shapely import wkt

tree_gdf = gpd.read_file(output_csv, crs="epsg:28992")
tree_gdf["location"] = tree_gdf["location"].apply(wkt.loads)
tree_gdf["convex_hull"] = tree_gdf["convex_hull"].apply(wkt.loads)
if use_concave:
    tree_gdf["concave_hull"] = tree_gdf["concave_hull"].apply(wkt.loads)
    tree_gdf.set_geometry("concave_hull", inplace=True)
else:
    tree_gdf.set_geometry("convex_hull", inplace=True)
tree_gdf.drop("geometry", axis=1, inplace=True)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, figsize=(10, 7))

tree_gdf.plot(ax=ax, color="green", alpha=0.8)
tree_gdf.set_geometry("location").plot(
    ax=ax, color="black", alpha=0.8, marker=".", markersize=5
)

plt.show()